# Impacting elastic bar on a rigid obstacle


## Prerequisites


### Jupyter notebooks reminder

A notebook is a sequence of "cells" that can be executed.


Each cell can handle either python code or markdown for comments.

* Edit a cell : Enter
* Execute a cell: Shift + Enter
* Run all cells : kernel menu (top of the page) --> Run all
* Delete cell : DD
* Add cell : Ctrl-mb
* Shortcuts reminder : Ctrl-m h
* List all magic commands : %lsmagic

More :  https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html#references

Warning : cells can be executed in any order but results and variables are persistent (until a call to %reset or kernel->restart)



### Import siconos components
you may need to set PYTHONPATH or sys.path if siconos python packages are not in standard places.

In [1]:
from numpy.linalg import norm
from siconos.kernel import LagrangianLinearTIDS, NewtonImpactNSL,\
    LagrangianLinearTIR, Interaction, NonSmoothDynamicalSystem, MoreauJeanOSI,\
    TimeDiscretisation, LCP, TimeStepping

from siconos.kernel import SimpleMatrix, getMatrix, SPARSE
import numpy as np

# I - Modeling : NonSmooth Dynamical Systems (NSDS) definition 
--> Dynamical systems, constraints, nonsmooth laws ...

In [2]:
# User-defined main parameters
nDof = 500  #degrees of freedom for the beam
t0 = 1e-8    #initial computation time
T = 0.0015                  # final computation time
h = 2e-7                # time step
position_init = 0.00005      # initial position
velocity_init =  -.1      # initial velocity
epsilon = 0.5#1e-1
theta = 1/2.0 + epsilon              # theta for MoreauJeanOSI integrator
#theta = 1.0
E = 210e9 # young Modulus
S = 0.000314 #  Beam Section 1 cm  for the diameter
#S=0.1
L = 1.0 # length of the  beam
l = L/nDof # length of an element
rho = 7800.0  # specific mass
#rho=1.0
g = 9.81 # Gravity
g=0.0

In [3]:
from scipy.sparse import csr_matrix

K = csr_matrix((nDof, nDof))
M = csr_matrix((nDof, nDof))


K[0,0] = 1.
K[0,1] = -1.
M[0,0] = 1/3.
M[0,1] = 1/6.

for i in range(1,nDof-1):
    K[i,i] = 2.
    K[i,i-1] = -1.
    K[i,i+1] = -1.
    M[i,i] = 2/3.
    M[i,i-1] = 1/6.
    M[i,i+1] = 1/6.


K[nDof-1,nDof-2] = -1.
K[nDof-1,nDof-1] = 1.
M[nDof-1,nDof-2] = 1/6.
M[nDof-1,nDof-1] = 1/3.

print(K.toarray())
print(M.toarray())


/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:7: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  import sys
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:8: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  if __name__ == '__main__':
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  # Remove the CWD from sys.path w

[[ 1. -1.  0. ...  0.  0.  0.]
 [-1.  2. -1. ...  0.  0.  0.]
 [ 0. -1.  2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  2. -1.  0.]
 [ 0.  0.  0. ... -1.  2. -1.]
 [ 0.  0.  0. ...  0. -1.  1.]]
[[0.33333333 0.16666667 0.         ... 0.         0.         0.        ]
 [0.16666667 0.66666667 0.16666667 ... 0.         0.         0.        ]
 [0.         0.16666667 0.66666667 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.66666667 0.16666667 0.        ]
 [0.         0.         0.         ... 0.16666667 0.66666667 0.16666667]
 [0.         0.         0.         ... 0.         0.16666667 0.33333333]]


/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:21: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:22: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:24: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [4]:
M= SimpleMatrix(nDof,nDof,SPARSE,nDof)
K= SimpleMatrix(nDof,nDof,SPARSE,nDof)
K.setValue(0,0, 1.*E*S/l)
K.setValue(0,1,-1.*E*S/l)
M.setValue(0,0, 1/3.*rho*S*l)
M.setValue(0,1, 1/6.*rho*S*l)

for i in range(1,nDof-1):
    K.setValue(i,i,2.*E*S/l)
    K.setValue(i,i-1,-1.*E*S/l)
    K.setValue(i,i+1,-1.*E*S/l)
    M.setValue(i,i,2/3.*rho*S*l)
    M.setValue(i,i-1,1/6.*rho*S*l)
    M.setValue(i,i+1,1/6.*rho*S*l)


K.setValue(nDof-1,nDof-2,-1.*E*S/l)
K.setValue(nDof-1,nDof-1, 1.*E*S/l)
M.setValue(nDof-1,nDof-2,1/6.*rho*S*l)
M.setValue(nDof-1,nDof-1,1/3.*rho*S*l)


In [5]:
q0 = np.full((nDof), position_init)
v0 = np.full((nDof), velocity_init)


In [7]:
bar = LagrangianLinearTIDS(q0,v0,M)
bar.setKPtr(K)
#bar.display()

weight = np.full((nDof),-g*rho*S/l)
bar.setFExtPtr(weight)


In [8]:
e=0.0

H = np.zeros((1,nDof))
H[0,0]=1.

nslaw = NewtonImpactNSL(e)
relation = LagrangianLinearTIR(H)
inter = Interaction(nslaw, relation)

In [9]:
# -------------
# --- Model ---
# -------------
impactingBar = NonSmoothDynamicalSystem(t0, T)

# add the dynamical system in the non smooth dynamical system
impactingBar.insertDynamicalSystem(bar);

# link the interaction and the dynamical system
impactingBar.link(inter,bar);


# II - Simulation definition

In [10]:

# ------------------
# --- Simulation ---
# ------------------

# -- (1) OneStepIntegrators --
OSI = MoreauJeanOSI(theta,0.5)

# -- (2) Time discretisation --
t = TimeDiscretisation(t0,h)

# -- (3) one step non smooth problem
osnspb = LCP()

s = TimeStepping(impactingBar, t,OSI,osnspb)

The model is now complete and ready to run

# III -  Running the simulation

In [11]:
k =0

N = int((T-t0)/h)
dataPlot = np.zeros((N+1, 5))

q = bar.q()
v = bar.velocity()
p = bar.p(1)
lambda_ = inter.lambda_(1)

# time loop
while s.hasNextEvent():
    s.computeOneStep()

    dataPlot[k, 0] = s.nextTime()
    dataPlot[k, 1] = q[0]
    dataPlot[k, 2] = v[0]
    dataPlot[k, 3] = p[0]/h
    dataPlot[k, 4] = lambda_[0]

    k += 1
    s.nextStep()

dataPlot.resize(k,5)
    

## IV - Post-processing

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline  
fig_size = [14, 14]
plt.rcParams["figure.figsize"] = fig_size

plt.subplot(411)
plt.title('position')
plt.plot(dataPlot[:, 0], dataPlot[:, 1])
plt.grid()
plt.subplot(412)
plt.title('velocity')
plt.plot(dataPlot[:, 0], dataPlot[:, 2])
plt.grid()
plt.subplot(413)
plt.plot(dataPlot[:, 0], dataPlot[:, 3])
plt.title('reaction')
plt.grid()
plt.subplot(414)
plt.plot(dataPlot[:, 0], dataPlot[:, 4])
plt.title('lambda')
plt.grid()


plt.show()